In [1]:
import pandas as pd
dataset = pd.read_json("../Datasets/dataset_humor_train.json", lines=True)
#conteo de clases
print("Total de ejemplos de entrenamiento")
print(dataset.klass.value_counts())
# Extracción de los textos en arreglos de numpy
X_train = dataset['text'].to_numpy()
# Extracción de las etiquetas o clases de entrenamiento
Y_train = dataset['klass'].to_numpy()

Total de ejemplos de entrenamiento
klass
0    6588
1    3812
Name: count, dtype: int64


In [2]:
import pandas as pd
dataset = pd.read_json("../Datasets/dataset_humor_test.json", lines=True)
#conteo de clases
print("Total de ejemplos de prueba")
print(dataset.klass.value_counts())
# Extracción de los textos en arreglos de numpy
X_test = dataset['text'].to_numpy()
# Extracción de las etiquetas o clases de entrenamiento
Y_test = dataset['klass'].to_numpy()

Total de ejemplos de prueba
klass
-1    5600
Name: count, dtype: int64


In [3]:
import re

def eliminar_emojis(texto):
    # Expresión regular que cubre la mayoría de emojis y pictogramas
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticonos
        "\U0001F300-\U0001F5FF"  # símbolos y pictogramas
        "\U0001F680-\U0001F6FF"  # transporte y mapas
        "\U0001F1E0-\U0001F1FF"  # banderas
        "\U00002702-\U000027B0"  # otros símbolos
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', texto)

In [4]:
import unicodedata
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import SnowballStemmer

_STOPWORDS = stopwords.words("english")  # agregar más palabras a esta lista si es necesario
stemmer = SnowballStemmer("english")
PUNCTUACTION = ";:,.\\-\"'/"
SYMBOLS = "()[]¿?¡!{}~<>|"
NUMBERS= "0123456789"
SKIP_SYMBOLS = set(PUNCTUACTION + SYMBOLS)
SKIP_SYMBOLS_AND_SPACES = set(PUNCTUACTION + SYMBOLS + '\t\n\r ')

def normaliza_texto(input_str,
                    punct=False,
                    accents=False,
                    num=False,
                    max_dup=2):
    """
        punct=False (elimina la puntuación, True deja intacta la puntuación)
        accents=False (elimina los acentos, True deja intactos los acentos)
        num= False (elimina los números, True deja intactos los acentos)
        max_dup=2 (número máximo de símbolos duplicados de forma consecutiva, rrrrr => rr)
    """
    
    nfkd_f = unicodedata.normalize('NFKD', input_str)
    n_str = []
    c_prev = ''
    cc_prev = 0
    for c in nfkd_f:
        if not num:
            if c in NUMBERS:
                continue
        if not punct:
            if c in SKIP_SYMBOLS:
                continue
        if not accents and unicodedata.combining(c):
            continue
        if c_prev == c:
            cc_prev += 1
            if cc_prev >= max_dup:
                continue
        else:
            cc_prev = 0
        n_str.append(c)
        c_prev = c
    texto = unicodedata.normalize('NFKD', "".join(n_str))
    texto = re.sub(r'(\s)+', r' ', texto.strip(), flags=re.IGNORECASE)
    return texto

def mi_preprocesamiento_factory(modo):
    """
    Devuelve una función de preprocesamiento y tokenización según el modo:
    - 'normalizacion'
    - 'normalizacion_stopwords'
    - 'normalizacion_stopwords_stem'
    """

    def preprocesamiento(texto):
        texto = texto.lower()
        texto = eliminar_emojis(texto)
        texto = re.sub(r"http\S+|www\S+|https\S+", "", texto)
        texto = re.sub(r"@\w+", "", texto)
        texto = normaliza_texto(texto)
        return texto

    def tokenizador(texto):
        tokens = word_tokenize(texto)
        if modo in ['normalizacion_stopwords', 'normalizacion_stopwords_stem']:
            tokens = [t for t in tokens if t not in _STOPWORDS and len(t) > 2]
        if modo == 'normalizacion_stopwords_stem':
            tokens = [stemmer.stem(t) for t in tokens]
        return tokens

    return preprocesamiento, tokenizador

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

class Vec_TFID:
    def __init__(self, modo_preproc='normalizacion'):
        self.vec_tfidf = None
        self.modo_preproc = modo_preproc

    def create_matriz_TFID(self, X_train, ngram_config, max_config):
        preproc, token = mi_preprocesamiento_factory(self.modo_preproc)
        self.vec_tfidf = TfidfVectorizer(
            analyzer="word",
            preprocessor=preproc,
            tokenizer=token,
            ngram_range=ngram_config,
            max_features=max_config
        )
        X_tfidf = self.vec_tfidf.fit_transform(X_train)
        return X_tfidf.toarray()

    def tranform_matriz_TFID(self, X_test):
        X_tfid = self.vec_tfidf.transform(X_test)
        return X_tfid.toarray()

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

class Vec_Count:
    def __init__(self, modo_preproc='normalizacion'):
        self.vec = None
        self.modo_preproc = modo_preproc

    def create_matriz_Count(self, X_train, ngram_config, max_config):
        preproc, token = mi_preprocesamiento_factory(self.modo_preproc)
        self.vec = CountVectorizer(
            analyzer="word",
            preprocessor=preproc,
            tokenizer=token,
            ngram_range=ngram_config,
            max_features=max_config
        )
        X_vec = self.vec.fit_transform(X_train)
        return X_vec.toarray()

    def transform_matriz_Count(self, X_test):
        X_vec = self.vec.transform(X_test)
        return X_vec.toarray()
